In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import lxml
import time

In [2]:
http_proxy  = "http://70.10.15.10:8080"
https_proxy = "http://70.10.15.10:8080"
# ftp_proxy   = "ftp://10.10.1.10:3128"

proxyDict = { 
              "http"  : http_proxy, 
              "https" : https_proxy 
#               "ftp"   : ftp_proxy
            }

In [3]:
# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와 
# 네이버 금융(http://finance.naver.com)에 넣어줌 

def get_url(code): 
#     code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code) 
    
    print("요청 URL = {}".format(url)) 
    return url 

In [4]:
# 마지막 페이지 조회
def get_last_page(url):
    response = requests.get(url, proxies=proxyDict)
#     print(response.content)
    soup = BeautifulSoup(response.content, 'html.parser')
    pgRR_url = soup.find("td",class_="pgRR").a['href']
    pgRR = int(pgRR_url[pgRR_url.rfind('=')+1:])
    print('마지막 page = {}'.format(pgRR))
    return pgRR

In [5]:
# 주식 데이터 가져오기
def get_stock_data(url, last_page, df):
    for page in range(1, last_page + 1):
        pg_url = '{url}&page={page}'.format(url=url, page=page)
        print(pg_url)
        response = requests.get(pg_url, proxies=proxyDict)
        revised_content = add_arrow_data(response.content)
        df_temp = pd.read_html(revised_content, header=0, index_col=None)
        print('{} page processing -------------------------'.format(page))
#         print(df_temp)
        df = df.append(df_temp)
        time.sleep(3)
    return df

In [6]:
# url = 'http://finance.naver.com/item/sise_day.nhn?code=079160'
# response = requests.get(url, proxies=proxyDict
# soup = BeautifulSoup(response.content, 'html.parser')
def add_arrow_data(content):
    soup = BeautifulSoup(content, 'html.parser')
    table = soup.find('table')
    
    # 화살표 헤더 추가
    temp_th = BeautifulSoup('<th>arrow</th>', "lxml").html.body.contents[0]
    table.tr.insert(4, temp_th)

    # 데이터 없는 행 보정
    tr_colspan = table.find_all(attrs={"colspan" : "7"})
    for index in range(len(tr_colspan)):
        tr_each = tr_colspan[index]
        tr_each['colspan'] = 8

    # 화살표 데이터 추가
    tr_all = table.find_all('tr')
    for index in range(1, len(tr_all)):
        tr_each = tr_all[index]
        if len(tr_each) != 15 :
            continue
        arrow = tr_each.find('img').get('alt')
        if arrow == '상승':
            arrow = 'UP'
        elif arrow == '하락':
            arrow = 'DOWN'
        else:
            arrow = ''
        temp_td = BeautifulSoup('<td>'+arrow+'</td>', "lxml").html.body.contents[0]
        tr_each.insert(4, temp_td)

    return soup.prettify()

In [7]:
# 신라젠의 일자데이터 url 가져오기 
item_name='CJ CGV'
item_code='079160' # CJ CGV 
url = get_url(item_code)
lastPage = get_last_page(url)

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=079160
마지막 page = 333


In [8]:
# 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame()

# 일자 데이터 조회
lastPage = 3
df = get_stock_data(url, lastPage, df)

# df.sort=False
df.head()

http://finance.naver.com/item/sise_day.nhn?code=079160&page=1
1 page processing -------------------------


c:\users\sds\appdata\local\programs\python\python36-32\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


http://finance.naver.com/item/sise_day.nhn?code=079160&page=2
2 page processing -------------------------
http://finance.naver.com/item/sise_day.nhn?code=079160&page=3
3 page processing -------------------------


,1,10,2,3,4,5,6,7,8,9,...,거래량,고가,날짜,다음,맨뒤,맨앞,시가,저가,전일비,종가
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,141717.0,66700.0,2018.05.31,NaN,NaN,NaN,66200.0,65200.0,300.0,66000.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,236562.0,67600.0,2018.05.30,NaN,NaN,NaN,66800.0,65000.0,1000.0,65700.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,232796.0,68600.0,2018.05.29,NaN,NaN,NaN,67300.0,66500.0,200.0,66700.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,281402.0,68700.0,2018.05.28,NaN,NaN,NaN,68400.0,66700.0,1400.0,66900.0


In [9]:
df.head()

,1,10,2,3,4,5,6,7,8,9,...,거래량,고가,날짜,다음,맨뒤,맨앞,시가,저가,전일비,종가
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,141717.0,66700.0,2018.05.31,NaN,NaN,NaN,66200.0,65200.0,300.0,66000.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,236562.0,67600.0,2018.05.30,NaN,NaN,NaN,66800.0,65000.0,1000.0,65700.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,232796.0,68600.0,2018.05.29,NaN,NaN,NaN,67300.0,66500.0,200.0,66700.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,281402.0,68700.0,2018.05.28,NaN,NaN,NaN,68400.0,66700.0,1400.0,66900.0
